In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "RA"
BARK_VAR = "PN"
FIA_SPCD = 351

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(0.075256, 0.94373, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,1798.000000,1.798000e+03,1798.0,1798.000000,1798.000000,1798.000000,1798.000000,1798.000000,1798.000000,1798.000000,...,95.000000,95.000000,1798.000000,1798.000000,1798.000000,1798.000000,1798.0,1798.000000,1798.000000,1798.000000
mean,633.651279,4.634763e+11,351.0,159.117475,0.312314,2.634360,11.535039,9.866463,0.432987,0.363115,...,0.407895,0.933684,2005.231368,7.732481,2015.231368,6.837041,10.0,1.423804,8.017241,257.965517
std,45.690025,7.784769e+10,0.0,48.573408,0.257128,2.101840,8.493190,4.679193,0.181566,0.153697,...,0.243677,0.564011,2.450143,2.075418,2.450143,2.122730,0.0,0.798949,5.944113,149.787094
min,505.000000,6.010158e+10,351.0,10.000000,0.000000,0.000000,1.000000,5.000000,0.050000,0.010000,...,0.050000,0.100000,2001.000000,1.000000,2011.000000,3.000000,10.0,0.000000,0.000000,0.000000
25%,609.000000,4.101040e+11,351.0,133.283900,0.100000,0.436332,5.000000,6.400000,0.300000,0.250000,...,0.250000,0.550000,2003.000000,6.000000,2013.000000,5.000000,10.0,1.000000,2.000000,133.000000
50%,612.000000,4.102030e+11,351.0,155.040000,0.250000,2.617994,9.000000,8.300000,0.400000,0.350000,...,0.400000,0.850000,2005.000000,8.000000,2015.000000,6.000000,10.0,1.000000,8.000000,247.000000
75%,618.000000,5.306028e+11,351.0,191.976000,0.500000,4.537856,15.000000,12.000000,0.550000,0.450000,...,0.550000,1.200000,2007.000000,9.000000,2017.000000,9.000000,10.0,2.000000,13.000000,388.000000
max,800.000000,5.308078e+11,351.0,336.516000,1.350000,6.283185,41.000000,40.900000,0.990000,0.900000,...,1.200000,3.000000,2009.000000,12.000000,2019.000000,11.000000,10.0,5.000000,22.000000,518.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [15:06<00:00,  2.21it/s, 255 steps of size 2.07e-02. a


Done.
Samples saved at ../../models/maicf/RA//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/RA//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [25:18<00:00,  1.32it/s, 255 steps of size 2.15e-02. a


Done.
Samples saved at ../../models/maicf/RA//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/RA//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/RA//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/RA//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [15:19<00:00,  2.18it/s, 255 steps of size 1.97e-02. a


Done.
Samples saved at ../../models/maicf/RA//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/RA//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [25:52<00:00,  1.29it/s, 255 steps of size 1.96e-02. a


Done.
Samples saved at ../../models/maicf/RA//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/RA//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [11:47<00:00,  2.83it/s, 255 steps of size 2.03e-02. a


Done.
Samples saved at ../../models/maicf/RA//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/RA//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [12:14<00:00,  2.72it/s, 255 steps of size 1.75e-02. a


Done.
Samples saved at ../../models/maicf/RA//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/RA//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/RA//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/RA//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [19:05<00:00,  1.75it/s, 255 steps of size 1.77e-02. a


Done.
Samples saved at ../../models/maicf/RA//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/RA//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [18:49<00:00,  1.77it/s, 255 steps of size 1.85e-02. a


Done.
Samples saved at ../../models/maicf/RA//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/RA//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
